# Setup Remote in Google Colab

+ Setup code-server for editting online
+ Setup ssh service for remoting and editting code
+ Using ngrok for binding code-server (port 9000) and ssh service (port 22)

## Mount Google Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

default_ws_dir="/content/drive/MyDrive/WorkSpace" #@param {type: "string"}

Mounted at /content/drive


## Params

In [2]:
# arguments
import argparse
parser = argparse.ArgumentParser(add_help=False)
parser.add_argument('--ws-dir', type=str, default=default_ws_dir)
args, _ = parser.parse_known_args()

# workspace
ws_dir=args.ws_dir

## Init script

In [3]:
# https://github.com/abhishekkrthakur/colabcode/blob/master/colabcode/code.py
# setup vs code server
# token input
token_file=f"{ws_dir}/data/.ngrok/token.txt" #@param {type: "string"}
pass_file=f"{ws_dir}/data/.ngrok/pass.txt" #@param {type: "string"}

id_rsa_pub_file=f"{ws_dir}/data/.ssh/id_rsa.pub" #@param {type: "string"}

extensions_dir=f"{ws_dir}/data/.vscode/extensions" #@param {type: "string"}
user_data_dir=f"{ws_dir}/data/.vscode/user_data" #@param {type: "string"}

In [4]:
!mkdir -p /a/b/c/d

## Read config

In [5]:
!mkdir -p $extensions_dir
!mkdir -p $user_data_dir

with open(token_file, "rt") as file: auth_token=file.read()

with open(pass_file, "rt") as file: password=file.read()
password = password.replace("\n", "")

with open(id_rsa_pub_file, "rt") as file: id_rsa_pub=file.read()

## Install code-server

In [6]:
# install code-server and start with port 9000
!curl -fsSL https://code-server.dev/install.sh | sh 2>&1 > /dev/null
!nohup bash -c "PASSWORD=\"$password\" code-server --port 9000 --user-data-dir=$user_data_dir --extensions-dir=$extensions_dir --disable-telemetry $ws_dir" &
for extension in ["ms-python.python", "ms-toolsai.jupyter", "mechatroner.rainbow-csv", "vscode-icons-team.vscode-icons"]:
    !code-server --install-extension $extension

######################################################################## 100.0%
nohup: appending output to 'nohup.out'
Installing extensions...
Installing extension 'ms-python.python'...
Extension 'ms-python.python' v2022.14.0 was successfully installed.
Installing extensions...
Extension 'ms-toolsai.jupyter' v2022.8.1002170547 is already installed. Use '--force' option to update to latest version or provide '@<version>' to install a specific version, for example: 'ms-toolsai.jupyter@1.2.3'.
Installing extensions...
Installing extension 'mechatroner.rainbow-csv'...
Extension 'mechatroner.rainbow-csv' v2.4.0 was successfully installed.
Installing extensions...
Installing extension 'vscode-icons-team.vscode-icons'...
Extension 'vscode-icons-team.vscode-icons' v11.16.0 was successfully installed.


## Install OpenSSH

In [7]:
# install, setup ssh and set public key
!apt-get install ssh 2>&1 > /dev/null
!service ssh restart
!mkdir -p ~/.ssh
!cat $id_rsa_pub_file > ~/.ssh/authorized_keys

 * Restarting OpenBSD Secure Shell server sshd
   ...done.


## Install ngrok and binding ports

In [8]:
# install pyngrok
!pip install -qqq pyngrok 2>&1 > /dev/null

# ngrok auth token
from pyngrok import ngrok, conf
list_regions = ["us", "en", "au", "vn"]

url, ssh_tunnel = None, None
for region in list_regions:
    try:
        conf.get_default().region = region
        ngrok.set_auth_token(auth_token)

        # bind with code-server: port 9000
        url = ngrok.connect(9000, "http")

        # bind with ssh: port 22
        ssh_tunnel = ngrok.connect(22, "tcp")
    except:
        pass
    
    # for
print(f"code-server: {url}")
print(f"ssh: {ssh_tunnel}")

code-server: NgrokTunnel: "http://bbf0-34-86-236-149.ngrok.io" -> "http://localhost:9000"
ssh: NgrokTunnel: "tcp://4.tcp.ngrok.io:16887" -> "localhost:22"


# End

In [9]:
print("Finsh setup!")

Finsh setup!
